In [ ]:
# Author: Haru
# Last editted: Feb 14, 2022

from sudachipy import tokenizer
from sudachipy import dictionary
import os
import re 
import nltk
import pandas as pd
import textspan
import pykakasi
import warnings
import MeCab
import unidic
import urllib
import jaconv
import re
import regex
import json
import pickle
# ignore depreciation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

In [ ]:
# CREATE YOMI DICTIONARY WITH MOJIKIBAN
def turn_into_list(text):
    new_list = text.split("・")
    return new_list

yomi_df = pd.read_csv ('/Users/harukatakagi/Desktop/EMPATH/mji.00601.csv', usecols= ['font','読み(参考)'])
yomi_df = yomi_df.rename(columns={'font': 'kanji_char', '読み(参考)': 'yomi'})
yomi_df = yomi_df[yomi_df['yomi'].notnull()]
yomi_df = yomi_df[yomi_df['kanji_char'] != '実装なし']

yomi_df['yomi'] = yomi_df.apply(lambda row: turn_into_list(row['yomi']), axis=1)
yomi_dictionary = dict(yomi_df.values)
del yomi_dictionary['𪕯']
del yomi_dictionary['𬼜']


In [ ]:
# CLEAN YOMI DICTIONARY

def has_kanji(text):
    p = regex.compile(r'\p{Script=Han}+')
    search_val = p.search(text)
    if search_val == None:
        return False
    else:
        return True

def has_kana(text):
    p = re.compile('[\u30A1-\u30FF]+')
    q = re.compile('[\uFF66-\uFF9F]+')
    search_val_p = p.search(text)
    search_val_q = q.search(text)
    if (search_val_p == None) and (search_val_q) == None:
        return False
    else:
        return True

def has_hiragana(text):
    p = re.compile('[\u3041-\u309F]+')
    search_val = p.search(text)
    if search_val == None:
        return False
    else:
        return True

for word in list(yomi_dictionary):
    if has_kanji(word) == False:
        del yomi_dictionary[word]
    elif (has_hiragana(word) == True) or (has_kana(word) == True):
        del yomi_dictionary[word]

In [ ]:
# MANUAL ADDS

yomi_dictionary['母']=['ボ', 'ボウ', 'モ', 'ム', 'はは', 'かあ']

In [ ]:
# YOMI WITHOUT OKURIGANA DICTIONARY
# Rule for 々?

def is_kana(text):
    converted_text = jaconv.hira2kata(text)
    if converted_text == text:
        return True
    else:
        return False

def is_hiragana(text):
    converted_text = conv.do(text)
    if converted_text == text:
        return True
    else:
        return False

kakasi = pykakasi.kakasi()
kakasi.setMode('J', 'H') # モードの設定：J(Kanji) to H(Hiragana)
conv = kakasi.getConverter()

count = 0
for thing in list(yomi_dictionary):
    count = count + 1
    print(str(count) + ' out of ' + str(len(yomi_dictionary)))
    yomi_list = yomi_dictionary[thing]
    new_yomi = []
    for element in yomi_list:
        if is_kana(element) == True:
            new_yomi = new_yomi + [element]
        else:
            
            url = "http://www.google.com/transliterate?"
            param = {'langpair':'ja-Hira|ja','text':element}
            paramStr = urllib.parse.urlencode(param)
            readObj = urllib.request.urlopen(url + paramStr)
            
            response = readObj.read()
            data = json.loads(response)
            fixed_data = json.loads(json.dumps(data[0], ensure_ascii=False))
            alt_yomi = fixed_data[1]
            for word in alt_yomi:
                if (thing == word):
                    new_yomi = new_yomi + [element]
                if (thing in word) and (is_hiragana(word[1:]) == True):
                    hira_to_drop = word[1:]
                    hira_to_drop_len = -len(hira_to_drop)
                    new_element = element[:hira_to_drop_len]
                    new_yomi = new_yomi + [new_element]
    new_yomi = list(dict.fromkeys(new_yomi))
    if '' in new_yomi:
        new_yomi.remove('')
    if (new_yomi == []):
        del yomi_dictionary[thing]
        continue
    for wee in new_yomi:
        if is_kana(wee) == True:
            continue
        wee_len = len(wee)
        for bee in new_yomi:
            if len(bee) <= wee_len:
                continue
            if wee == bee[0:wee_len]:
                new_yomi.remove(bee)     
    yomi_dictionary[thing]  = new_yomi

print('DONE')
del yomi_dictionary['々']


In [ ]:
# SAVE DICTIONARY

with open('yomi_dictionary_no_okurigana.pkl', 'wb') as f:
    pickle.dump(yomi_dictionary, f)

In [ ]:
yomi_dictionary['空']